In [1]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16
from PIL import *

2022-03-31 18:06:48.916058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-31 18:06:48.916096: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Path to folders with training data
dog_path = Path("./ex_files/Ch05/training_data/dogs")
not_dog_path = Path("./ex_files/Ch05/training_data/not_dogs")

images = []
labels = []

# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)
    
    # Convert the image to a numpy array
    image_array = image.img_to_array(img)
    
    # Add the image to the list of images
    images.append(image_array)
    
    # For each 'not dot' image, the expected value should be 0
    labels.append(0)
    
# Load all the dog images
for img in dog_path.glob("*.png"):
    # Load the image from the disk
    img = image.load_img(img)
    
    # Convert the image to a numpy array
    image_array = image.img_to_array(img)
    
    # Add the image to the list of images
    images.append(image_array)
    
    # For each 'dog' image, the expected value should be 1
    labels.append(1)
    
# Create a single numpy array with all the images we loaded
x_train =  np.array(images)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
pretrained_nn = vgg16.VGG16(weights="imagenet",
                            include_top=False,
                            input_shape=(64,64,3)
                           )

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)


# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values
joblib.dump(y_train, "y_train.dat")

2022-03-31 18:06:54.308782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-31 18:06:54.308829: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-31 18:06:54.308864: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lucas-Inspiron-5566): /proc/driver/nvidia/version does not exist
2022-03-31 18:06:54.309560: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-31 18:06:55.263885: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3355443

['y_train.dat']

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib

# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

# Create a model and add layers

model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train the model
model.fit(x_train,
          y_train,
         epochs=10,
         shuffle=True
         )

# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")

f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")

Epoch 1/10
2/2 [==============================] - 1s 9ms/step - loss: 8.8437 - accuracy: 0.6552
Epoch 2/10
2/2 [==============================] - 0s 13ms/step - loss: 0.6335 - accuracy: 0.9310
Epoch 3/10
2/2 [==============================] - 0s 9ms/step - loss: 0.8392 - accuracy: 0.9310
Epoch 4/10
2/2 [==============================] - 0s 8ms/step - loss: 0.3615 - accuracy: 0.9655
Epoch 5/10
2/2 [==============================] - 0s 7ms/step - loss: 0.5291 - accuracy: 0.9483
Epoch 6/10
2/2 [==============================] - 0s 8ms/step - loss: 1.7604e-09 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 15ms/step - loss: 7.9985e-17 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 14ms/step - loss: 2.2407e-05 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 8ms/step - loss: 1.7184e-10 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 10ms/step - loss: 3.1100e-11 - accuracy: 1.0000
